In [1]:
# library(sceasy)
# library(reticulate)

# # Convert .h5ad to Seurat object
# sceasy::convertFormat(
#   "data/merfish.h5ad", 
#   from = "anndata", 
#   to = "seurat",
#   outFile = "data/merfish.rds"
# )

In [2]:
# table output folder
out_folder='tab_out'
if (!dir.exists(out_folder)) {
  dir.create(out_folder, recursive = TRUE)
} else {
  cat("Folder already exists:", out_folder, "\n")
}

Folder already exists: tab_out 


In [3]:
library(Seurat)
library(DOT)
library(caret)
library(dplyr)
mop= readRDS('data/merfish.rds')


Attaching SeuratObject

Seurat v4 was just loaded with SeuratObject v5; disabling v5 assays and
validation routines, and ensuring assays work in strict v3/v4
compatibility mode

Loading required package: ggplot2

Loading required package: lattice


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [6]:
alpha <- 0.25
results <- list()

# Get unique slices
slices <- unique(mop$slice_id)
# Initialize an empty data.frame to store results
results_df <- data.frame(
  Nb_Gene = integer(),
  Slice_ID = character(),
  Accuracy = numeric(),
  F1_Score = numeric(),
  stringsAsFactors = FALSE
)

# Loop over the number of genes
for (nb_gene in c(50, 100, 150, 200)) {
    
  for (slice in slices) {
      print(slice)
    # Subset data for the target and reference
    mop_sub <- subset(mop, subset = slice_id == slice)
    mop_ref <- subset(mop, subset = slice_id != slice)

    # Introduce noise to the target
    beta_values <- 1 + runif(ncol(mop_sub), min = -alpha, max = alpha)
    mop_sub@assays$RNA@counts <- t(t(as.matrix(mop_sub@assays$RNA@counts) * beta_values))
    mop_sub_sub <- mop_sub[1:nb_gene, ]

    # Get coordinates
    coord_sub_sub <- mop_sub_sub@meta.data[, c("x", "y")]

    # Apply DOT
    dot_srt <- setup.srt(srt_data = mop_sub_sub, srt_coords = coord_sub_sub)
    dot_ref <- setup.ref(ref_data = mop_ref, ref_annotations = mop_ref$subclass)
    dot <- create.DOT(dot_srt, dot_ref)
    dot <- run.DOT.highresolution(dot, verbose = FALSE)

    # Compute accuracy
    predictions <- factor(colnames(dot@weights)[apply(dot@weights, 1, which.max)])
    true_labels=mop_sub_sub$subclass
    missing_classes <- setdiff(levels(predictions), levels(true_labels))
    true_labels <- factor(true_labels, levels = c(levels(true_labels), missing_classes))
    common_levels <- union(levels(true_labels), levels(predictions))
    true_labels <- factor(true_labels, levels = common_levels)
    predictions <- factor(predictions, levels = common_levels)
      
    accuracy<- sum(true_labels == predictions) / length(predictions)
    conf_matrix <- table(Predicted = predictions, Actual = true_labels)
    precision <- diag(conf_matrix) / colSums(conf_matrix)  # Precision for each class
    recall <- diag(conf_matrix) / rowSums(conf_matrix)  # Recall for each class
    f1_scores <- 2 * (precision * recall) / (precision + recall)
    f1_scores[is.na(f1_scores)]=0
    f1_score <- mean(f1_scores)
    print(accuracy)
    print(f1_score)
      
    results_df <- results_df %>%
      add_row(
        Nb_Gene = nb_gene,
        Slice_ID = slice,
        Accuracy = accuracy,
        F1_Score = f1_score
      )           
  }}


[1] "mouse2_slice31"


Computing spatial radius

Warning message:
“did not converge in 10 iterations”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 500000)”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 500000)”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”


[1] 0.8178694
[1] 0.6693368
[1] "mouse2_slice160"


Computing spatial radius

Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 500000)”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”

[1] 0.7906209
[1] 0.7026297
[1] "mouse2_slice109"


...

[1] 0.9542211
[1] 0.9366338
[1] "mouse2_slice261"


Computing spatial radius

Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 303500)”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 247750)”
Warning message:
“did not conve

[1] 0.9318564
[1] 0.8649878
[1] "mouse2_slice20"


Computing spatial radius

Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“Quick-TRANSfer stage steps exceeded maximum (= 500000)”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”
Warning message:
“did not converge in 10 iterations”


[1] 0.9253731
[1] 0.7923473


In [8]:
write.csv(results_df,paste0(out_folder,'/CellAnnotation_MERFISH_dot.csv'))